## Client Collaboration
First, let's set up our client. Fill in the same username you used in the previous notebook to generate your certificate.

In [ ]:
import securexgboost as xgb
from Utils import *

In [ ]:
# TODO: fill in the IP of the enclave server
server_ip = "104.211.33.210"
server_port = "50052" 
remote_addr = server_ip + ":" + server_port 

In [ ]:
# Get variables from previous notebooks
%store -r 

## Data Transfer
Next, centralize the data by send your training and test data to the person controlling the enclave server. The `transfer_data()` function below is a Python wrapper around the command line `scp` function.

In [ ]:
transfer_data(enc_training_data, server_ip)
transfer_data(enc_test_data, server_ip)

## Client Initialization and Authentication
Once everyone has sent their data to the server, we can initialize our client.

In [ ]:
# Run this cell to initialize your client

# TODO: fill out `client_list`
clients = []
xgb.init_client(user_name=username, client_list=clients, sym_key_file=KEY_FILE, priv_key_file=PUB_KEY, cert_file=CERT_FILE, remote_addr=remote_addr)

Before we perform any computation, we want to attest that the remote enclave on the untrusted server has loaded the proper code. Secure XGBoost provides this functionality through the `attest()` API.

In [ ]:
# Verify that the enclave has been set up correctly
xgb.attest()

## Collaborative Pipeline Orchestration
Once we've authenticated the enclave, we can begin making requests to the enclave server. MC<sup>2</sup> enables users to make requests through a Python API, but will only execute requests once all users in the collaboration have submitted the same request. Consequently, users must submit the exact same requests in the exact order if they want to collaboratively compute. 

Let's first prepare for training by loading everyone's encrypted training data within the enclave. Parties transferred their data to the central server's `/home/mc2/risecamp/mc2/tutorial/central/` directory. Each party with username `usr` named their training data `user_train.enc`.

In [ ]:
central_data_dir = "/home/mc2/risecamp/mc2/tutorial/central/"

Fill in the paths to each party's training data. MC<sup>2</sup>'s `DMatrix()` function takes in a dictionary 

`{"username1": "/path/to/user1/data", "username2": "/path/to/user2/data"}`. 

For example if the collaboration has two users, `user1` and `user2`, our dictionary argument to `DMatrix()` would be 

`{"user1": central_data_dir + "user1_train.enc", "user2": central_data_dir + "user2_train.enc}`

In [ ]:
# TODO: fill in usernames and training data paths
dtrain = xgb.DMatrix({"username1": "/path/to/user1/data", "username2": "/path/to/user2/data"})

In [ ]:
# Set parameters
params = {
        "tree_method": "hist",
        "n_gpus": "0",
        "objective": "binary:logistic",
        "min_child_weight": "1",
        "gamma": "0.1",
        "max_depth": "3",
        "verbosity": "1" 
}
num_rounds = 5
booster = xgb.train(params, dtrain, num_rounds)

In [ ]:
# Load everyone's test data

# TODO: fill in usernames and test data paths
dtest1 = xgb.DMatrix({"chief": central_data_dir + "chief_test.enc"})
dtest2 = xgb.DMatrix({"chester": central_data_dir + "chester_test.enc"})
# dtest3 = xgb.DMatrix({username: enc_test_data})

In [ ]:
# Allow everyone to get their predictions
enc_preds, num_preds = booster.predict(dtest1, decrypt=False)
booster.predict(dtest2, decrypt=False)
# enc_preds, num_preds = booster.predict(dtest, decrypt=False)

In [ ]:
# Decrypt our predictions
preds = booster.decrypt_predictions(enc_preds, num_preds)
print(preds)